In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [2]:
x_2=inputs[1]
d_in=inputs.shape[1]
d_out=2


In [3]:
torch.manual_seed(123)
w_query=torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False)
w_key=torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False)
w_value=torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False) 


In [4]:
print("Query:",w_query)


Query: Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]])


test with the indivisual word


In [5]:
query_2=x_2 @ w_query
key_2=x_2 @ w_key
value_2=x_2 @ w_value
print(query_2)
print(key_2)
print(value_2)

tensor([-1.1729, -0.0048])
tensor([-0.1142, -0.7676])
tensor([0.4107, 0.6274])


Let's do this with the Whole Input


In [6]:
query=inputs @ w_query
keys=inputs @ w_key
values= inputs @ w_value

print(query)

tensor([[-1.1686,  0.2019],
        [-1.1729, -0.0048],
        [-1.1438, -0.0018],
        [-0.6339, -0.0439],
        [-0.2979,  0.0535],
        [-0.9596, -0.0712]])


In [7]:
keys_2=keys[1]
attn_Score=query_2 @ keys_2
print(attn_Score) 

tensor(0.1376)


In [8]:

attn_Score=query @ keys.T
print(attn_Score) 

tensor([[ 0.0740, -0.0216,  0.0126, -0.1230,  0.6250, -0.4498],
        [ 0.2172,  0.1376,  0.1730, -0.0491,  0.7616, -0.3809],
        [ 0.2098,  0.1320,  0.1665, -0.0489,  0.7408, -0.3725],
        [ 0.1458,  0.1061,  0.1254, -0.0118,  0.4384, -0.1919],
        [ 0.0175, -0.0071,  0.0017, -0.0321,  0.1580, -0.1153],
        [ 0.2240,  0.1642,  0.1935, -0.0161,  0.6667, -0.2888]])


In [9]:
normalized_attn_Score=torch.nn.functional.softmax(attn_Score,dim=1)
print(normalized_attn_Score)

tensor([[0.1667, 0.1515, 0.1568, 0.1369, 0.2893, 0.0988],
        [0.1689, 0.1560, 0.1616, 0.1294, 0.2912, 0.0929],
        [0.1691, 0.1564, 0.1619, 0.1305, 0.2876, 0.0945],
        [0.1710, 0.1643, 0.1675, 0.1461, 0.2291, 0.1220],
        [0.1684, 0.1643, 0.1658, 0.1603, 0.1938, 0.1475],
        [0.1708, 0.1609, 0.1657, 0.1344, 0.2659, 0.1023]])


In [10]:
d_k=keys.shape[-1]
attn_weights_2=torch.nn.functional.softmax(attn_Score/d_k**2,dim=1)
print(attn_weights_2)

tensor([[0.1684, 0.1644, 0.1658, 0.1603, 0.1933, 0.1477],
        [0.1692, 0.1658, 0.1673, 0.1583, 0.1938, 0.1457],
        [0.1691, 0.1658, 0.1673, 0.1585, 0.1931, 0.1462],
        [0.1683, 0.1666, 0.1675, 0.1618, 0.1811, 0.1547],
        [0.1672, 0.1662, 0.1665, 0.1651, 0.1732, 0.1617],
        [0.1690, 0.1665, 0.1677, 0.1592, 0.1888, 0.1487]])


Calculating the Context Vector

In [11]:
context_vector=attn_weights_2 @ values
print(context_vector)

tensor([[0.2861, 0.3929],
        [0.2865, 0.3933],
        [0.2865, 0.3931],
        [0.2867, 0.3901],
        [0.2867, 0.3880],
        [0.2867, 0.3921]])


We will Create a class implementation of the Self Attention machanism with trainable weights


In [12]:
import torch.nn as nn

class SelfAttensionMachanism_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.w_query=nn.Parameter(torch.randn(d_in,d_out))
        self.w_key=nn.Parameter(torch.randn(d_in,d_out))
        self.w_value=nn.Parameter(torch.randn(d_in,d_out))


    def forward(self,inputs):
        keys=inputs @ self.w_key
        query=inputs @ self.w_query
        values=inputs @ self.w_value

        attn_score=query @ keys.T
        attn_weights=torch.nn.functional.softmax(attn_score/keys.shape[-1]**0.5,dim=-1)

        context_vector=attn_weights @ values
        return context_vector    

In [13]:
Am_v1=SelfAttensionMachanism_v1(d_in,d_out)
context_vector=Am_v1(inputs)
print(context_vector)

tensor([[0.1278, 0.2577],
        [0.1365, 0.2556],
        [0.1364, 0.2556],
        [0.1294, 0.2624],
        [0.1295, 0.2595],
        [0.1310, 0.2618]], grad_fn=<MmBackward0>)


In [14]:
import torch.nn as nn

class SelfAttensionMachanism_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.w_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value=nn.Linear(d_in,d_out, bias=qkv_bias)


    def forward(self,inputs):
        keys=self.w_key(inputs)
        query=self.w_query(inputs)
        values=self.w_value(inputs)

        attn_score=query @ keys.T
        attn_weights=torch.nn.functional.softmax(attn_score/keys.shape[-1]**0.5,dim=-1)

        context_vector=attn_weights @ values
        return context_vector    

In [15]:
Am_v2=SelfAttensionMachanism_v2(d_in,d_out)
context_vector=Am_v2(inputs)
print(context_vector)

tensor([[-0.4496,  0.4982],
        [-0.4462,  0.4957],
        [-0.4462,  0.4956],
        [-0.4478,  0.4971],
        [-0.4468,  0.4963],
        [-0.4479,  0.4971]], grad_fn=<MmBackward0>)


Causal Self Attention Machanism


In [16]:
queries=Am_v2.w_query(inputs)
key=Am_v2.w_key(inputs)

attention_score=queries @ key.T

attention_weight=torch.nn.functional.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
print(attention_weight)

tensor([[0.1708, 0.1646, 0.1647, 0.1665, 0.1681, 0.1654],
        [0.1723, 0.1579, 0.1587, 0.1690, 0.1818, 0.1603],
        [0.1722, 0.1579, 0.1587, 0.1690, 0.1819, 0.1603],
        [0.1696, 0.1616, 0.1621, 0.1681, 0.1755, 0.1630],
        [0.1703, 0.1596, 0.1603, 0.1688, 0.1796, 0.1615],
        [0.1701, 0.1617, 0.1622, 0.1680, 0.1750, 0.1631]],
       grad_fn=<SoftmaxBackward0>)


In [17]:
context_length=attention_weight.shape[0]
simple_mask=torch.tril(torch.ones(context_length,context_length))
print(simple_mask)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [18]:
masked_simple = attention_weight * simple_mask
print(masked_simple)

tensor([[0.1708, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1723, 0.1579, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1722, 0.1579, 0.1587, 0.0000, 0.0000, 0.0000],
        [0.1696, 0.1616, 0.1621, 0.1681, 0.0000, 0.0000],
        [0.1703, 0.1596, 0.1603, 0.1688, 0.1796, 0.0000],
        [0.1701, 0.1617, 0.1622, 0.1680, 0.1750, 0.1631]],
       grad_fn=<MulBackward0>)


In [19]:
rows_sum=masked_simple.sum(dim=1,keepdim=True)
masked_attention_weights=masked_simple/rows_sum
print(masked_attention_weights)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5217, 0.4783, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3524, 0.3230, 0.3246, 0.0000, 0.0000, 0.0000],
        [0.2564, 0.2443, 0.2450, 0.2542, 0.0000, 0.0000],
        [0.2031, 0.1904, 0.1911, 0.2013, 0.2141, 0.0000],
        [0.1701, 0.1617, 0.1622, 0.1680, 0.1750, 0.1631]],
       grad_fn=<DivBackward0>)


In [20]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
masked = attention_weight.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0.1708,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1723, 0.1579,   -inf,   -inf,   -inf,   -inf],
        [0.1722, 0.1579, 0.1587,   -inf,   -inf,   -inf],
        [0.1696, 0.1616, 0.1621, 0.1681,   -inf,   -inf],
        [0.1703, 0.1596, 0.1603, 0.1688, 0.1796,   -inf],
        [0.1701, 0.1617, 0.1622, 0.1680, 0.1750, 0.1631]],
       grad_fn=<MaskedFillBackward0>)


In [21]:
attention_weight=torch.nn.functional.softmax(masked/key.shape[-1]**0.5,dim=-1)
print(attention_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5025, 0.4975, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3355, 0.3321, 0.3323, 0.0000, 0.0000, 0.0000],
        [0.2508, 0.2493, 0.2494, 0.2505, 0.0000, 0.0000],
        [0.2004, 0.1989, 0.1989, 0.2002, 0.2017, 0.0000],
        [0.1671, 0.1661, 0.1661, 0.1668, 0.1676, 0.1662]],
       grad_fn=<SoftmaxBackward0>)


In [22]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(p=0.5)
example=torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [23]:

print(dropout(attention_weight))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6647, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4987, 0.0000, 0.5010, 0.0000, 0.0000],
        [0.0000, 0.3977, 0.3979, 0.4003, 0.4034, 0.0000],
        [0.3341, 0.3322, 0.0000, 0.0000, 0.3353, 0.3325]],
       grad_fn=<MulBackward0>)


In [24]:
import torch.nn as nn
class CausalSelfAttention(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.w_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout=torch.nn.Dropout(p=0.1)

    def forward(self,inputs):
        keys=self.w_key(inputs)
        query=self.w_query(inputs)
        values=self.w_value(inputs)

        attention_score=query @ keys.T
        attention_weight=torch.nn.functional.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        context_length=attention_weight.shape[0]
        simple_mask=torch.tril(torch.ones(context_length,context_length))
        masked_simple = attention_weight * simple_mask
        rows_sum=masked_simple.sum(dim=1,keepdim=True)
        masked_attention_weights=masked_simple/rows_sum
        masked_attention_weights*=values
        mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
        masked = attention_weight.masked_fill(mask.bool(),-torch.inf)
        attention_weight=torch.nn.functional.softmax(masked/key.shape[-1]**0.5,dim=-1)
        
        

        return self.dropout(attention_weight)

In [25]:
Am_v2=CausalSelfAttention(3,6)
context_vector=Am_v2(inputs)
print(context_vector)

tensor([[1.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5600, 0.5511, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3742, 0.3682, 0.3687, 0.0000, 0.0000, 0.0000],
        [0.2781, 0.2762, 0.2764, 0.2805, 0.0000, 0.0000],
        [0.2229, 0.2194, 0.2196, 0.2229, 0.0000, 0.0000],
        [0.0000, 0.1832, 0.1833, 0.1870, 0.1882, 0.1852]],
       grad_fn=<MulBackward0>)


In [26]:
batch=torch.stack((inputs,inputs,inputs,inputs,inputs,inputs,inputs,inputs),dim=0)
print(batch.shape)

torch.Size([8, 6, 3])


Causal Self Attention with the batch sizes

In [27]:
import torch.nn as nn
class CausalSelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,qkv_bias=False):
        super().__init__()
        self.w_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout=torch.nn.Dropout(p=dropout)
        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))
    
    def forward(self,inputs):
        b,num_tokens,d_in=inputs.shape      
        keys=self.w_key(inputs)
        query=self.w_query(inputs)
        values=self.w_value(inputs)

        attention_score=query @ keys.transpose(1,2)
        attn_Score.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens].bool(),-torch.inf)
        
        attention_weight=torch.nn.functional.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        attention_weight=self.dropout(attention_weight)
        context_vector=attention_weight @values
        return context_vector

In [28]:
torch.manual_seed(123)
context_length=batch.shape[1]
ca=CausalSelfAttention_v2(d_in,d_out,context_length,0.0)
context_vector=ca(batch)
print(context_vector)

tensor([[[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
    

Multihead attention Machanism


In [29]:
class MultiheadAttensionMachanism(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bais=False):
        super().__init__()
        self.heads=nn.ModuleList([
             CausalSelfAttention_v2(d_in,d_out,context_length,dropout,qkv_bais )
             for _ in range(num_heads)
        ])

    def forward(self,x ):
        return torch.cat([head(x) for head in self.heads],dim=-1)    

In [30]:
torch.manual_seed(123)
m1=MultiheadAttensionMachanism(d_in,d_out,context_length,0.1,3,False)
context_vector=m1(batch)
print(context_vector)

tensor([[[-0.5191, -0.0904,  0.5650,  0.3897,  0.4649,  0.2185],
         [-0.5915, -0.1200,  0.4928,  0.3737,  0.5907,  0.2765],
         [-0.5915, -0.1199,  0.4938,  0.3240,  0.5176,  0.2352],
         [-0.3656, -0.0533,  0.5638,  0.3856,  0.5933,  0.2779],
         [-0.4538, -0.0808,  0.4739,  0.3267,  0.4188,  0.2054],
         [-0.5888, -0.1201,  0.5641,  0.3882,  0.5037,  0.2243]],

        [[-0.5930, -0.1168,  0.4707,  0.3309,  0.5913,  0.2768],
         [-0.5915, -0.1200,  0.5649,  0.3898,  0.3950,  0.1895],
         [-0.5233, -0.0920,  0.5649,  0.3896,  0.5907,  0.2765],
         [-0.5886, -0.1195,  0.5638,  0.3856,  0.5933,  0.2779],
         [-0.4538, -0.0808,  0.3526,  0.2706,  0.5923,  0.2775],
         [-0.5888, -0.1201,  0.5641,  0.3882,  0.4634,  0.2187]],

        [[-0.5930, -0.1168,  0.5650,  0.3897,  0.4988,  0.2215],
         [-0.5915, -0.1200,  0.5649,  0.3898,  0.4654,  0.2187],
         [-0.5233, -0.0920,  0.4708,  0.3309,  0.5907,  0.2765],
         [-0.4182, -0

Implementing Multihead Attention with Weight Split

In [31]:
class MultiheadAttensionMachanism_v2(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bais=False):
        super().__init__()
        assert( d_out%num_heads==0),\
            "d_out must be Divisible by numeber of Heads"
        self.d_out=d_out
        self.d_in=d_in
        self.num_head=num_heads
        self.head_dim=d_out//num_heads
        self.w_query=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.w_key=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.w_value=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)

        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))


    def forward(self,x):
        b,num_tokens,d_in=x.shape

        keys=self.w_key(x)
        queries=self.w_query(x)
        value=self.w_value(x)

        keys=keys.view(b,num_tokens,self.num_head,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_head,self.head_dim)
        values=value.view(b,num_tokens,self.num_head,self.head_dim)

        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)

        attn_scores=queries @ keys.transpose(2,3)
        
        mask_bool=self.mask.bool()[:num_tokens,:num_tokens]
        attn_scores=masked_fill_(mask_bool,-torch.inf)

        attn_weights=torch.softmax(attn_scores/key.shape[-1]**0.5,dim=-1)
        attn_weights=self.dropout(attn_weights)

        context_vec=(attn_weights @ value).transpose(1,2)

        context_vec=context_vec.contiguous().view(b,num_tokens,self.d_out)
        context_vec=self.out_proj(context_vec)

        return context_vec

In [32]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [33]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 6])


In [34]:
class MultiheadAttensionMachanism_v2(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bais=False):
        super().__init__()
        assert( d_out%num_heads==0),\
            "d_out must be Divisible by numeber of Heads"
        self.d_out=d_out
        self.d_in=d_in
        self.num_head=num_heads
        self.head_dim=d_out//num_heads
        self.w_query=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.w_key=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.w_value=nn.Linear(d_in,d_out,bias=qkv_bais)
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)

        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))


    def forward(self,x):
        b,num_tokens,d_in=x.shape

        keys=self.w_key(x)
        queries=self.w_query(x)
        value=self.w_value(x)

        keys=keys.view(b,num_tokens,self.num_head,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_head,self.head_dim)
        values=value.view(b,num_tokens,self.num_head,self.head_dim)

        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)

        attn_scores=queries @ keys.transpose(2,3)
        
        mask_bool=self.mask.bool()[:num_tokens,:num_tokens]
        attn_scores=masked_fill_(mask_bool,-torch.inf)

        attn_weights=torch.softmax(attn_scores/key.shape[-1]**0.5,dim=-1)
        attn_weights=self.dropout(attn_weights)

        context_vec=(attn_weights @ value).transpose(1,2)

        context_vec=context_vec.contiguous().view(b,num_tokens,self.d_out)
        context_vec=self.out_proj(context_vec)

        return context_vec

LLM Architecture Block


In [35]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

Dummy GPT Model

In [36]:
import torch
import torch.nn as nn


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

Step 1: Tokenize All the text

In [37]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


Step 2: Apply the Layer Normalization on the inputs

In [38]:
torch.manual_seed(123)
batch_example=torch.randn(2,5)
layer=nn.Sequential(nn.Linear(5,6),nn.ReLU())
out=layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


In [39]:
mean=out.mean(dim=-1,keepdim=True)
var=out.var(dim=-1,keepdim=True)
print("Mean of the Inputs is:\n",mean)
print("Variance of the inputs is:\n",var)

Mean of the Inputs is:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance of the inputs is:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


In [40]:
out_norm=(out-mean)/torch.sqrt(var)
mean=out_norm.mean(dim=-1,keepdim=True)
var=out_norm.var(dim=-1,keepdim=True)
print("Normalized Values:\n",out_norm)
print("Mean of Normalized values:\n",mean)
print("Variance of Normalized Values:\n",var)


Normalized Values:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean of Normalized values:
 tensor([[9.9341e-09],
        [5.9605e-08]], grad_fn=<MeanBackward1>)
Variance of Normalized Values:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [41]:
torch.set_printoptions(sci_mode=False)


print("Normalized Values:\n",out_norm)
print("Mean of Normalized values:\n",mean)
print("Variance of Normalized Values:\n",var)


Normalized Values:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean of Normalized values:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Variance of Normalized Values:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


Making the class of this code


In [42]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var  = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [43]:
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


GELU Activation Function

In [44]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [45]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [46]:
print(GPT_CONFIG_124M["emb_dim"])


768


In [47]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768) #A
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


ShortCut Connection

In [48]:
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

    def forward(self, x):
        for layer in self.layers:
            # Compute the output of the current layer
            layer_output = layer(x)
            # Check if shortcut can be applied
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x


In [49]:
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123) # specify random seed for the initial weights for reproducibility
model_without_shortcut = ExampleDeepNeuralNetwork(
layer_sizes, use_shortcut=False
)

In [50]:
layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123) # specify random seed for the initial weights for reproducibility
model_with_shortcut = ExampleDeepNeuralNetwork(
layer_sizes, use_shortcut=True
)

In [51]:
def print_gradients(model, x):
    # Forward pass
    output = model(x)
    target = torch.tensor([[0.]])

    # Calculate loss based on how close the target
    # and output are
    loss = nn.MSELoss()
    loss = loss(output, target)
    
    # Backward pass to calculate the gradients
    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
            # Print the mean absolute gradient of the weights
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [52]:
print_gradients(model_without_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.00020173590746708214
layers.1.0.weight has gradient mean of 0.0001201116101583466
layers.2.0.weight has gradient mean of 0.0007152042235247791
layers.3.0.weight has gradient mean of 0.0013988739810883999
layers.4.0.weight has gradient mean of 0.00504964729771018


In [53]:
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.22169791162014008
layers.1.0.weight has gradient mean of 0.20694102346897125
layers.2.0.weight has gradient mean of 0.32896995544433594
layers.3.0.weight has gradient mean of 0.2665732204914093
layers.4.0.weight has gradient mean of 1.3258541822433472


In [54]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [55]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x) 
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [56]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768) #A
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


Entire GPT Model

In [57]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        ) 

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [58]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1381,  0.0077, -0.1963,  ..., -0.0222, -0.1060,  0.1717],
         [ 0.3865, -0.8408, -0.6564,  ..., -0.5163,  0.2369, -0.3357],
         [ 0.6989, -0.1829, -0.1631,  ...,  0.1472, -0.6504, -0.0056],
         [-0.4290,  0.1669, -0.1258,  ...,  1.1579,  0.5303, -0.5549]],

        [[ 0.1094, -0.2894, -0.1467,  ..., -0.0557,  0.2911, -0.2824],
         [ 0.0882, -0.3552, -0.3527,  ...,  1.2930,  0.0053,  0.1898],
         [ 0.6091,  0.4702, -0.4094,  ...,  0.7688,  0.3787, -0.1974],
         [-0.0612, -0.0737,  0.4751,  ...,  1.2463, -0.3834,  0.0609]]],
       grad_fn=<UnsafeViewBackward0>)


Total Number Of Trainable Parameters

In [59]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [60]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [61]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 124,412,160


Memory Requirement for this Model

In [62]:
total_size_bytes = total_params * 4 #A
total_size_mb = total_size_bytes / (1024 * 1024) #B
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


Generating The New Word

In [63]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [64]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [65]:
model.eval() #A
out = generate_text_simple(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


In [66]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue
